In [ ]:
import pandas as pd
import os, sys
import time
from tqdm import tqdm 
import importlib
sys.path.append(os.path.abspath(".."))
from utils import images_clf



In [2]:
importlib.reload(images_clf)
from utils.images_clf import check_pic_url, download_image

In [9]:
df=pd.read_csv("../A_preprocess\data_jo_3_processed\listings_jo_paris2024_processed1.csv")
print(len(df))

C:\Users\yeliu\AppData\Local\Temp\ipykernel_28264\742478674.py:1: DtypeWarning: Columns (78,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("../A_preprocess\data_jo_3_processed\listings_jo_paris2024_processed1.csv")


74579


# test:

## stratifid sampling


In [ ]:
df=df.drop_duplicates(subset=['host_id'])
print(len(df))#-5K

N_total=2000
strata = (
    df
    .groupby(["is_changed", "has_host_about", "lang"])
    .size()
    .reset_index(name="N_s")
)
# 每个类别按照原df比例在smaple中应该拥有的数量 = 按比例 * 总数
strata["n_sample"] = (strata["N_s"] / strata["N_s"].sum() * N_total).round().astype(int)
print(strata,"\n")


sampled_list = []
for _, row in strata.iterrows():
    cond = (
        (df["is_changed"] == row["is_changed"]) &
        (df["has_host_about"] == row["has_host_about"]) &
        (df["lang"] == row["lang"])
    )
    subset = df[cond]

    # 如果某层数量不足，直接全取
    n = min(row["n_sample"], len(subset))

    sampled = subset.sample(n=n, random_state=42)
    sampled_list.append(sampled)

sampled_df = pd.concat(sampled_list, ignore_index=True)
print(sampled_df,"\n")

id_url_df=sampled_df.copy()[:][['host_id','host_picture_url']]
id_url_list= list(zip(id_url_df["host_id"], id_url_df["host_picture_url"]))
print(len(id_url_list))

53414
   is_changed  has_host_about         lang    N_s  n_sample
0       False               0      no_text  23019       862
1       False               1           en   5844       219
2       False               1           fr   8203       307
3       False               1      no_text     98         4
4       False               1  other_langs    423        16
5        True               0      no_text  10744       402
6        True               1           en   1896        71
7        True               1           fr   3010       113
8        True               1      no_text     23         1
9        True               1  other_langs    154         6 

      Unnamed: 0                   id  \
0          70606  1048307549455709351   
1          53957   889964538720137624   
2          55318   905706794059511080   
3           6052              6599882   
4          52030   866332686709711350   
...          ...                  ...   
1996       84812  1123977884705743498   
1997

## download:

In [ ]:
importlib.reload(images_clf)
from utils.images_clf import download_images_batch

# results = download_images_batch(id_url_list, out_dir='images_raw', max_workers=8)

success = [p for p in results.values() if p]
fail = [u for u, p in results.items() if not p]#无效url/下载失败！

print(f"[SUCCES] {len(success)} 张")
print(f"[FAIL] {len(fail)} 张：")

[WARNING] 无效 URL: https://a0.muscache.com/im/pictures/user/User-34961932/original/65ec51af-e1af-4157-bdaf-d28b92a340c6.jpeg?aki_policy=profile_x_medium


[WARNING] 无效 URL: https://a0.muscache.com/im/pictures/user/User-559574902/original/199d8914-fb04-4292-b9b8-9a21fe021ed2.jpeg?aki_policy=profile_x_medium


[WARNING] 无效 URL: https://a0.muscache.com/im/pictures/user/149760ba-c918-4ce3-8670-421de17f841e.jpg?aki_policy=profile_x_medium


[WARNING] 无效 URL: https://a0.muscache.com/im/pictures/user/331fcb46-9021-406a-9d37-8e396cdba838.jpg?aki_policy=profile_x_medium


[WARNING] 无效 URL: https://a0.muscache.com/im/users/26081843/profile_pic/1440313903/original.jpg?aki_policy=profile_x_medium


...

[WARNING] 无效 URL: https://a0.muscache.com/im/pictures/user/User-574630696/original/70bb3589-1ba1-4ff3-be3e-248af546fbaf.jpeg?aki_policy=profile_x_medium


[WARNING] 无效 URL: https://a0.muscache.com/im/pictures/user/83de5532-a161-42e9-852a-a04c754d0e30.jpg?aki_policy=profile_x_medium


[WARNING] 无效 URL: https://a0.muscache.com/im/pictures/user/a0c4fbcf-676c-477f-8196-436862275a21.jpg?aki_policy=profile_x_medium


[INFO] host 438828 pic 已存在，跳过： images_raw\438828.jpg



 [SUCCES] downloaded 2001 : 673.04 sec!

[SUCCES] 1987 张
[FAIL] 14 张：


In [8]:
results

{438828: 'images_raw\\438828.jpg',
 474546: 'images_raw\\474546.jpg',
 429406: 'images_raw\\429406.jpg',
 464019: 'images_raw\\464019.jpg',
 228508: 'images_raw\\228508.jpg',
 48733: 'images_raw\\48733.jpg',
 108444: 'images_raw\\108444.jpg',
 81840: 'images_raw\\81840.jpg',
 532468: 'images_raw\\532468.jpg',
 107514: 'images_raw\\107514.jpg'}

## split:

In [ ]:

# 合并多个分层变量
strat_col = sampled_df[["is_change","has_host_about","lang_group"]].astype(str).agg("_".join, axis=1)

# 拆成 test / train
train_df, test_df = train_test_split(
    sampled_df,
    test_size=0.15,
    stratify=strat_col,
    random_state=42
)

# 剩下的 pool = not in train/test
pool_df = sampled_df.loc[
    ~sampled_df.index.isin(train_df.index) & ~sampled_df.index.isin(test_df.index)
]


In [52]:
sampled_df[["is_changed","has_host_about","lang"]].value_counts(dropna=False)

is_changed  has_host_about  lang       
False       0               no_text        862
True        0               no_text        402
False       1               fr             307
                            en             219
True        1               fr             113
                            en              71
False       1               other_langs     16
True        1               other_langs      6
False       1               no_text          4
True        1               no_text          1
Name: count, dtype: int64

In [7]:
# strat = sampled_df[["is_changed","has_host_about","lang"]].apply(lambda x: "_".join([str(v) for v in x]), axis=1)
#restart!
strat.value_counts()

False_0_no_text        862
True_0_no_text         402
False_1_fr             307
False_1_en             219
True_1_fr              113
True_1_en               71
False_1_other_langs     16
True_1_other_langs       6
False_1_no_text          4
True_1_no_text           1
Name: count, dtype: int64

In [ ]:
from sklearn.model_selection import train_test_split

test_ratio = 0.15
train_ratio = 0.35
pool_ratio = 0.50   # 不用直接用，最后自动得到

# Step 1：POOL vs REST
pool_df, rest_df = train_test_split(
    sampled_df,
    test_size=pool_ratio,
    # stratify=strat,
    random_state=42
)
# print(f"POOL vs REST : {len(pool_df)}, {len(rest_df)}")


train_ratio_updated = train_ratio / (1 - pool_ratio)  # ≈ 0.4118

train_df, test_df = train_test_split(
    rest_df,
    test_size=train_ratio_updated,
    # stratify=strat,
    random_state=42
)
print(f"TEST vs TRAIN vs POOL :{len(test_df)}, {len(train_df)}, {len(pool_df)}")


POOL vs REST : 1000, 1001
TRAIN vs REST : 300, 701


## copy:

In [ ]:
importlib.reload(images_clf)
from utils.images_clf import spli

# application :

In [1]:
import pandas as pd
import os, sys
import time
from tqdm import tqdm 
import importlib
sys.path.append(os.path.abspath(".."))
from utils import images_clf

In [2]:
importlib.reload(images_clf)
from utils.images_clf import stratified_sampling, download_images_batch

In [3]:
df=pd.read_csv("../A_preprocess\data_jo_3_processed\listings_jo_paris2024_processed1.csv")
sampled_df, id_url_list=stratified_sampling(df, groupby=["is_changed", "has_host_about", "lang"], N_total=2000)


*******************STRATIFIED SAMPLING***************************
METHODS:
regroupe df by is_changed,has_host_about,lang;
sampling proportionally 2000 pics;
=> get a list of set (host_id, host_picture_id)

===============INPUT INFO=================
len before deduplication by host_id:74579
after:53414

strata :
    is_changed  has_host_about         lang    N_s  n_sample
0       False               0      no_text  23019       862
1       False               1           en   5844       219
2       False               1           fr   8203       307
3       False               1      no_text     98         4
4       False               1  other_langs    423        16
5        True               0      no_text  10744       402
6        True               1           en   1896        71
7        True               1           fr   3010       113
8        True               1      no_text     23         1
9        True               1  other_langs    154         6

len SAMPLE/'id_url_list':

C:\Users\yeliu\AppData\Local\Temp\ipykernel_25116\4024380113.py:1: DtypeWarning: Columns (78,80) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("../A_preprocess\data_jo_3_processed\listings_jo_paris2024_processed1.csv")


,host_id,host_picture_url,is_changed,has_host_about,lang
0,41925864,https://a0.muscache.com/im/users/41925864/prof...,False,0,no_text
1,6853322,https://a0.muscache.com/im/pictures/user/039a5...,False,0,no_text
2,410087584,https://a0.muscache.com/im/pictures/user/ad305...,False,0,no_text
3,3394353,https://a0.muscache.com/im/users/3394353/profi...,False,0,no_text
4,143767086,https://a0.muscache.com/im/pictures/user/User-...,False,0,no_text
